# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,45.39,67,78,10.11,NZ,1690347432
1,1,skadovs'k,46.1161,32.9112,77.41,69,19,20.20,UA,1690347432
2,2,port-aux-francais,-49.3500,70.2167,38.01,96,88,30.24,TF,1690347433
3,3,adamstown,-25.0660,-130.1015,72.12,77,98,36.80,PN,1690347433
4,4,waitangi,-43.9535,-176.5597,46.81,94,99,23.71,NZ,1690347433


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df["Max Temp"] < 90) & (city_data_df["Max Temp"] > 70) & (city_data_df["Cloudiness"] < 25) & (city_data_df["Wind Speed"] < 15)]

# Drop any rows with null values
ideal_weather_cities_complete = ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities_complete

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,hamilton,39.1834,-84.5333,75.22,84,15,4.38,US,1690347188
8,8,kill devil hills,36.0307,-75.6760,81.99,84,0,5.75,US,1690347434
11,11,olyokminsk,60.3736,120.4264,89.13,33,0,6.40,RU,1690347434
14,14,el porvenir,13.7617,-87.3458,78.13,75,16,6.49,HN,1690347435
15,15,kingston,17.9970,-76.7936,85.15,84,20,9.22,JM,1690347435
...,...,...,...,...,...,...,...,...,...,...
532,532,robore,-18.3333,-59.7500,71.42,45,0,3.60,BO,1690347552
537,537,bouarfa,32.5300,-1.9500,73.00,35,0,9.31,MA,1690347553
539,539,sebring,27.4956,-81.4409,78.04,87,0,1.01,US,1690347554
542,542,homeland park,34.4707,-82.6707,75.72,87,0,0.00,US,1690347554


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_complete[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\vasqu\AppData\Local\Temp\ipykernel_23456\182853606.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
7,hamilton,US,39.1834,-84.5333,84,
8,kill devil hills,US,36.0307,-75.6760,84,
11,olyokminsk,RU,60.3736,120.4264,33,
14,el porvenir,HN,13.7617,-87.3458,75,
15,kingston,JM,17.9970,-76.7936,84,
...,...,...,...,...,...,...
532,robore,BO,-18.3333,-59.7500,45,
537,bouarfa,MA,32.5300,-1.9500,35,
539,sebring,US,27.4956,-81.4409,87,
542,homeland park,US,34.4707,-82.6707,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
kill devil hills - nearest hotel: Mariner Days Inn & Suites
olyokminsk - nearest hotel: Колос
el porvenir - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
isla mujeres - nearest hotel: Villa La Bella
severnoye - nearest hotel: No hotel found
mount pleasant - nearest hotel: Old Village Post House
darwin - nearest hotel: Mantra Pantanas Darwin
saint-pierre - nearest hotel: Tropic Hotel
zhanibek - nearest hotel: No hotel found
kununurra - nearest hotel: Kununurra Country Club Resort
uturoa - nearest hotel: Hawaiki Nui hotel
mattoon - nearest hotel: US Grant Motel
saint-philippe - nearest hotel: Le Baril
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
talnakh - nearest hotel: Талнах
semirom - nearest hotel: No hotel found
upper grand lagoon - nearest hotel: Hampton Inn Panama City Beach
portland - nearest hotel: the Hoxton
fort bragg - nearest hotel: Airborne Inn Lodging
san patrici

,City,Country,Lat,Lng,Humidity,Hotel Name
7,hamilton,US,39.1834,-84.5333,84,North Vista Manor
8,kill devil hills,US,36.0307,-75.6760,84,Mariner Days Inn & Suites
11,olyokminsk,RU,60.3736,120.4264,33,Колос
14,el porvenir,HN,13.7617,-87.3458,75,No hotel found
15,kingston,JM,17.9970,-76.7936,84,Aphrodite Hotel
...,...,...,...,...,...,...
532,robore,BO,-18.3333,-59.7500,45,Lajas
537,bouarfa,MA,32.5300,-1.9500,35,Climat du Maroc مناخ المغرب
539,sebring,US,27.4956,-81.4409,87,Residence Inn by Marriott Sebring
542,homeland park,US,34.4707,-82.6707,87,Home2 Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols = ["City", "Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)